In [1]:
#imports
import pandas as pd 
import numpy as np

In [2]:
# Load the Excel file
file_path = "DataLab2.xlsx" 

# Read the first 501 to 1009 rows and first 11 columns
df = pd.read_excel(file_path, header=0, skiprows=range(1, 501), usecols=range(11))

# Transform 'PL' column to create 'Losses' column
df["Losses"] = df["PL"] * (-1)

# extract the years
df["Year"] = df["Date"].dt.year.astype(int)

# Display the first few rows (for testing)
df.head()


Date   PL  VaRBHS      VaREWMA        VaRn        VaRt       VaRPot  \
0 2006-12-26 -150  1115.0  1147.464693  777.535296  986.872911  1391.726372   
1 2006-12-28  150  1115.0  1144.195603  778.321836  987.104816  1391.726372   
2 2006-12-29  -20  1115.0  1117.460475  778.334975  987.885905  1391.726372   
3 2006-12-31   30  1115.0  1085.901247  777.503184  989.976454  1391.726372   
4 2007-01-01   30  1115.0  1053.756229  777.645897  990.797723  1391.726372   

        ESEWMA         ESn          ESt        ESPot  Losses  Year  
0  2544.116901  897.215494  3209.017239  1924.333523     150  2006  
1  2540.137912  898.017553  3189.914672  1924.333523    -150  2006  
2  2506.631070  898.020953  3203.255498  1924.333523      20  2006  
3  2468.406961  897.126264  3261.771712  1924.333523     -30  2006  
4  2431.278879  897.263546  3270.873551  1924.333523     -30  2007

In [3]:
# Backtesting VAR

# encode exceptions for each VAR
var_indices = list(range(2,7)) # these are the columns indices for the VAR

# to get all the names of the headers for the VAR so we can use these when encoding
var_names = df.columns[var_indices] 

# here we create the headers for the exceptions, 0 no exception, 1 is.
ex_names = [f'{var_name}_exception' for var_name in var_names] 

# we zip the lists just to be able to loop over both
var_ex_names = list(zip(var_names, ex_names)) 

# here we encode a violation (exception) as a 1, otherwise 0
for var_name, ex_name in var_ex_names:
    df[ex_name] = (df[var_name] < df["Losses"]).astype(int) 

# Get indices of the exception columns (for easy retrieval later)
ex_indices = [df.columns.get_loc(ex_name) for ex_name in ex_names]

# the indices of the ES
es_indices = list(range(7,11))




In [4]:
# now to actually doing the tests for each year, alpha=0.99 was used in the first exercise for the vars, so we use that here
# first create a python dictionary for all the years, so we extract each row (with relevant data) for each year
# I also add the ES estimates since this will be useful later

years_data = {}
loss_index = [df.columns.get_loc("Losses")]

# here we create all the necessary indices
var_ex_es_loss_indices = var_indices + es_indices +loss_index+ ex_indices


for row, year in enumerate(df["Year"]):
    if year == 2006: # we skip year 2006 since we don't have enough data there
        continue

    # access all the data
    data = df.iloc[row, var_ex_es_loss_indices]
    
    if year in years_data:
        years_data[year].append(data)
    else:
        years_data[year] = [data]

# convert each year into a separate dataframe for easy analysis
for year in years_data:
    years_data[year] = pd.DataFrame(years_data[year])

print(years_data[2007])
# now we have a python dict with each year with the corresponding losses for that year, and the VARs for those years, 
#exceptions in order aswell inside each year, and ES for each year and day of that year after 2006 since we append.

     VaRBHS      VaREWMA         VaRn         VaRt       VaRPot       ESEWMA  \
4    1115.0  1053.756229   777.645897   990.797723  1391.726372  2431.278879   
5    1115.0  1021.701152   777.393080   993.867845  1391.726372  2391.877505   
6    1115.0   990.624063   777.083717   994.978159  1391.726372  2355.183490   
7    1115.0   966.503238   777.332311   993.984109  1391.726372  2326.891025   
8    1115.0   943.122243   775.856105   996.059063  1391.726372  2300.318478   
..      ...          ...          ...          ...          ...          ...   
251  2720.0  5446.739118  1850.330910  1872.782072  3923.705072  8667.750829   
252  2720.0  5281.329790  1851.518976  1880.739916  3923.705072  8417.587184   
253  2720.0  5140.982010  1854.690014  1898.367178  3923.705072  8194.339146   
254  2720.0  5025.876542  1891.472008  1899.165832  3923.705072  7982.628781   
255  2720.0  5636.548968  1892.561404  1911.666403  3923.705072  8916.984522   

             ESn           ESt        E

In [5]:
# now we will actually be doing the tests on each of the VARs
# since we only really care about underestimating losses, we will do one sided tests

#will be using the binomial distribution from scipy
from scipy.stats import binom


In [6]:
#defining a function for the kupiec test
def kupiec(ex_list: list, alpha=0.99, significance_level=0.05) -> str:
    # Total number of observations (days)
    n = len(ex_list)
    
    # Number of violations (exceptions should be 1 for a violation, 0 otherwise)
    k = sum(ex_list)
    
    # The expected probability of a violation under the null hypothesis
    p = 1 - alpha

    # Handle the case with 0 violations explicitly for clarity
    if k == 0:
        p_value = 1.0
    else:
        # For k >= 1, compute the p-value as the probability of seeing at least k violations
        p_value = 1 - binom.cdf(k - 1, n, p)
    
    # Return True if we reject the null hypothesis (p-value is less than the significance level)
    test = p_value < significance_level 
    if test:
        return "Reject"
    else:
        return "Accept"
    

In [7]:


# function for basel
def basel(ex_list: list, alpha = 0.99) -> str:
    n = len(ex_list)
    p = 1-alpha
    k = sum(ex_list)  # Count exceptions

    # Compute the cutoffs dynamically
    green_yellow = int(binom.ppf(0.95, n, p))  # 95% cutoff
    yellow_red = int(binom.ppf(0.99, n, p))  # 99% cutoff

    # Assign traffic light zone
    if k <= green_yellow:
        return "Green"
    elif green_yellow < k <= yellow_red:
        return "Yellow"
    else:
        return "Red"




In [8]:
# function for christofferson
from scipy.stats import chi2
def christofferson(ex_list: list, significance_level = 0.05) -> str:
    ex_list = list(ex_list)
    
    # initialize transition counts
    n_00 = n_01 = n_10 = n_11 = 0

    for i in range(1, len(ex_list)):  # Start at index 1 to check previous day
        prev, curr = ex_list[i - 1], ex_list[i]
        if prev == 0 and curr == 0:
            n_00 += 1
        elif prev == 0 and curr == 1:
            n_01 += 1
        elif prev == 1 and curr == 0:
            n_10 += 1
        elif prev == 1 and curr == 1:
            n_11 += 1

    # Calculate transition probabilities
    pi_01 = n_01 / (n_00 + n_01) if (n_00 + n_01) > 0 else 0  
    pi_00 = 1 - pi_01 # the complement
    
    pi_11 = n_11 / (n_10 + n_11) if (n_10 + n_11) > 0 else 0
    pi_10 = 1 - pi_11

    # compute L1
    L1 = pi_00**n_00 * pi_01**n_01 * pi_10**n_10 * pi_11**n_11 # chat says this might cause underflow errors but I think it is fine
    
    
    n_1 = sum(ex_list) # number of violations, look at the video
    n = len(ex_list) # length of list
    n_0 = n - n_1 # number of zeros
    pi_0 = n_1/n # expected probability
    pi_1 = n_1/n # or equivalently, just the complement

    #compute L0
    L0 = pi_0**n_0 * pi_1**n_1

    # compute the test
    LR = -2*(np.log(L0) - np.log(L1))if L1 > 0 and L0 > 0 else np.inf
    
    chi2_critical = chi2.ppf(1-significance_level, df=1)
    test_rule = LR > chi2_critical
    
    if test_rule:
        return "Reject"
    else:
        return "Accept"

In [9]:
# here we will be performing each of the tests

# create a dictionary keeping track of the test results, for each year, on each var (very complicated structure but works wonders)
test_data_var = {
    year: 
               {
        var_name:
                    {
    "Kupiec":None, 
    "Basel":None,
    "Christofferson":None
                    }
    for var_name in var_names
               }
    for year in years_data
            }


In [10]:
# loop over all the years, extract the exception columns, and perform each test on each encoded exception
for year, year_dataframe in years_data.items():

    #this gives all the exception columns for that year
    exceptions_columns = year_dataframe[ex_names] 

    #var_ex_names is a zipped list with the var and corresponding exceptions column name
    for var_name, ex_name in var_ex_names: 
        
        # the violations for that current type of var for that year
        ex_list = exceptions_columns[ex_name] 

         # here we do the kupiec test and store everything for that test, that var, that year
        test_data_var[year][var_name]["Kupiec"] = kupiec(ex_list)

        # here we do the basel test
        test_data_var[year][var_name]["Basel"] = basel(ex_list)

        # here we do the christofferson test
        test_data_var[year][var_name]["Christofferson"] = christofferson(ex_list)
        


In [11]:
# convert the data for nice visualization
year_with_dfs_var = {year: pd.DataFrame.from_dict(data, orient='index') for year, data in test_data_var.items()}

# Display each DataFrame
for year, year_df in year_with_dfs_var.items():
    print(f"\nData for {year}:")
    print(year_df)

# Accept Kupiec means VaR estimate is good
# Basel are the different zones
# Christofferson, Rejecting means States dependent


Data for 2007:
         Kupiec  Basel Christofferson
VaRBHS   Reject    Red         Reject
VaREWMA  Accept  Green         Reject
VaRn     Reject    Red         Reject
VaRt     Reject    Red         Reject
VaRPot   Accept  Green         Reject

Data for 2008:
         Kupiec   Basel Christofferson
VaRBHS   Reject  Yellow         Reject
VaREWMA  Reject  Yellow         Reject
VaRn     Reject     Red         Reject
VaRt     Reject     Red         Reject
VaRPot   Accept   Green         Reject


In [15]:
# Backtesting ES 
# Now we are moving on to ES estimates, we will in essence build the same kind of dataframe structure but do it for ES instead

#the names of the columns for ES
es_names = df.columns[es_indices]

test_data_es = {year: {es_name:None for es_name in es_names} for year in years_data}

# list with var and ES for easy retrieval later, there doesn't seem to be any data for ESBHS so I guess we just skip that. 
ex_es_names = list(zip(ex_names[1:],es_names)) # if that data is added I can fix it later by removing the 1 and fixing the columns indices above

#redefining alpha for readability
alpha = 0.99

from scipy.stats import t
# here we define the Acerbi-Szekely
def acerbi_szekely(L, ES, I, alpha=0.99):

    # Compute the Z-score using the Acerbi-Szekely formula
    M = len(L)  # Number of observations
    Z = - (1 / (M * (1 - alpha))) * np.sum((L * I) / ES) + 1

    # Estimate degrees of freedom (ν) using a Student-t fit on loss data
    estimated_v, _, _ = t.fit(L)
    v = round(estimated_v)  # Round ν to nearest integer

    # Define the Monte Carlo critical values table
    critical_values = {
        3: -0.82,  # 5% significance level
        5: -0.74,
        10: -0.71,
        100: -0.70,
        np.inf: -0.70  # Normal distribution case
    }

    # Select the closest ν from the table
    selected_v = min(critical_values.keys(), key=lambda x: abs(x - v))
    critical_value = critical_values[selected_v]

    # Interpretation
    if Z > critical_value:
        interpretation = "ES underestimates risk (too optimistic)."
    elif Z < critical_value:
        interpretation = "ES overestimates risk (too conservative)."
    else:
        interpretation = "ES is well-calibrated."

    return Z, selected_v, critical_value, interpretation


In [16]:
# loop over all the years, extract the indicator columns, and perform each test on each encoded exception

for year, year_dataframe in years_data.items():
    # here we extract the losses for that year
    losses = year_dataframe["Losses"]
    
    #this gives all the indicator columns for that year (exceptions essentially), still needed for ES since this is indicator
    indicator_columns = year_dataframe[ex_names] 

    # here we extract the es columns
    es_columns = year_dataframe[es_names]

    #var_ex_es_names is a zipped list with the var, corresponding exceptions, es column name
    for ex_name, es_name in ex_es_names: 
        
        # the violations for that current type of var for that year
        indicator = indicator_columns[ex_name] 

        es = es_columns[es_name]

         # here we do the kupiec test and store everything for that test, that var, that year
        test_data_es[year][es_name] = acerbi_szekely(losses,es,indicator)


In [17]:

# Convert test_data_es into a dictionary of DataFrames
year_with_dfs_es = {year: pd.DataFrame.from_dict(data, orient='index') for year, data in test_data_es.items()}

# Display each DataFrame
for year, year_df in year_with_dfs_es.items():
    print(f"\nExpected Shortfall Data for {year}:")
    print(year_df)



Expected Shortfall Data for 2007:
                0  1     2                                          3
ESEWMA  -0.466064  3 -0.82   ES underestimates risk (too optimistic).
ESn    -18.461961  3 -0.82  ES overestimates risk (too conservative).
ESt     -1.305906  3 -0.82  ES overestimates risk (too conservative).
ESPot   -1.166971  3 -0.82  ES overestimates risk (too conservative).

Expected Shortfall Data for 2008:
               0  1     2                                          3
ESEWMA -1.048678  3 -0.82  ES overestimates risk (too conservative).
ESn    -6.577423  3 -0.82  ES overestimates risk (too conservative).
ESt     0.110515  3 -0.82   ES underestimates risk (too optimistic).
ESPot  -0.502134  3 -0.82   ES underestimates risk (too optimistic).


In [ ]:
# Answers to the questions:

In [19]:
'''
(a) Do the same methods perform well (or not so well) for the different sample years, 2007/2008?

Based on the Kupiec, Basel, and Christoffersen tests, the performance of the VaR models varies between 2007 and 2008.

**2007 Findings:**
- VaREWMA and VaRPot performed well in 2007:
  - Basel Green Zone (suggesting a reasonable number of exceptions).
  - Kupiec Test Accepted (suggesting correct frequency of violations).

- VaRBHS, VaRn, and VaRt performed poorly:
  - Basel Red Zone (indicating excessive violations).
  - Kupiec Test Rejected (showing too many or too few exceptions).

- Christoffersen rejected all models, meaning exceptions were clustered rather than independent.

**2008 Findings:**
- VaRPot continued to perform well:
  - Basel Green Zone (consistent performance).
  - Kupiec Test Accepted.
- VaREWMA declined in performance:
  - Basel Yellow Zone (higher violations than expected).
- VaRBHS, VaRn, and VaRt continued to perform poorly:
  - Basel Red Zone.
  - Kupiec Rejected.

- Christoffersen again rejected all models, confirming exception clustering.

**Conclusion for VaR Models:**
- VaRPot is the most stable model across both years, maintaining a Green Basel rating.
- VaREWMA showed deterioration in 2008, indicating instability.
- VaRBHS, VaRn, and VaRt consistently failed, with excessive violations and clustering.

The Acerbi-Szekely test evaluates whether the Expected Shortfall (ES) estimates are well-calibrated.
A well-calibrated ES should have a Z-score close to 0, meaning it neither underestimates nor overestimates risk.
If Z < critical value, ES is too optimistic (dangerous, underestimates tail risk).
If Z > critical value, ES is too conservative (overestimates risk).

**2007 Findings:**
- **ESEWMA underestimated risk (too optimistic)**, meaning its ES values were too low.
- **ESn, ESt, and ESPot overestimated risk (too conservative)**, suggesting they were too pessimistic in their risk estimates.
- **ESn had the most extreme conservatism**, which could be inefficient for capital allocation.
- **ESt and ESPot were also too conservative**, overestimating tail risk.

**2008 Findings:**
- **ESEWMA and ESn overestimated risk (too conservative)**, meaning their ES values were higher than necessary.
- **ESt and ESPot underestimated risk (too optimistic)**, meaning they were not conservative enough in extreme loss scenarios.
- **ESt was particularly unreliable**, as it switched from conservative in 2007 to optimistic in 2008.

**Conclusion for ES Models:**
- **ESEWMA was the most stable model**, as it had only slight deviations in either direction.
- **ESn was consistently too conservative**, which might lead to excessive capital reserves, but makes sure the banks don't go into bankrupcy.
- **ESt was inconsistent**, switching from conservative to optimistic.
- **ESPot also showed inconsistency**, being conservative in 2007 but optimistic in 2008.
- **Best ES model: ESEWMA**, as it had the best balance between risk conservatism and model stability.
'''




'\n(a) Do the same methods perform well (or not so well) for the different sample years, 2007/2008?\n\nBased on the Kupiec, Basel, and Christoffersen tests, the performance of the VaR models varies between 2007 and 2008.\n\n**2007 Findings:**\n- VaREWMA and VaRPot performed well in 2007:\n  - Basel Green Zone (suggesting a reasonable number of exceptions).\n  - Kupiec Test Accepted (suggesting correct frequency of violations).\n\n- VaRBHS, VaRn, and VaRt performed poorly:\n  - Basel Red Zone (indicating excessive violations).\n  - Kupiec Test Rejected (showing too many or too few exceptions).\n\n- Christoffersen rejected all models, meaning exceptions were clustered rather than independent.\n\n**2008 Findings:**\n- VaRPot continued to perform well:\n  - Basel Green Zone (consistent performance).\n  - Kupiec Test Accepted.\n- VaREWMA declined in performance:\n  - Basel Yellow Zone (higher violations than expected).\n- VaRBHS, VaRn, and VaRt continued to perform poorly:\n  - Basel Red Zo

In [18]:
'''
(b) Which model would you pick for VaR and which would you pick for ES? Please motivate your choices.

**Best VaR Model: VaRPot**
- VaRPot consistently performed well in both 2007 and 2008.
  - Basel Green Zone in both years (indicating an appropriate number of violations).
  - Kupiec Test Accepted, meaning the number of VaR violations matched the expected probability.
- Although Christoffersen rejected all models (indicating clustered violations), VaRPot still outperformed others.
- VaRPot was more stable than VaREWMA, which dropped from Green in 2007 to Yellow in 2008.
- Other models (VaRBHS, VaRn, VaRt) performed poorly:
  - Basel Red Zones in both years, meaning excessive violations.
  - Kupiec Test Rejected, suggesting they either underestimated or overestimated risk.
  - Christoffersen Test Rejected, confirming violation clustering.

**Final Choice for VaR: VaRPot** (Reliable across years, Basel Green, Kupiec Accepted).

**Best ES Model: ESn**
- **From a regulator’s perspective, being too conservative is preferable to being too optimistic.**
- **ESn was the most conservative model**, significantly overestimating risk in both 2007 and 2008.
- **While this might be inefficient from a capital allocation standpoint, it ensures sufficient capital buffers.**
- **ESEWMA and ESPot were more balanced, but they showed small deviations in different directions.**
- **ESt was highly inconsistent**, underestimating risk in 2008.
- **ESPot also lacked consistency**, switching from conservative to optimistic between years.

**Final Choice for ES: ESn** (Most conservative, ensures sufficient risk coverage from a regulatory viewpoint).

**Final Selection Summary:**
- **Best VaR Model:** VaRPot (Consistent, Basel Green, Kupiec Accepted).
- **Best ES Model:** ESn (Most conservative, ensuring capital adequacy from a regulatory perspective).
'''

'\n(b) Which model would you pick for VaR and which would you pick for ES? Please motivate your choices.\n\n**Best VaR Model: VaRPot**\n- VaRPot consistently performed well in both 2007 and 2008.\n  - Basel Green Zone in both years (indicating an appropriate number of violations).\n  - Kupiec Test Accepted, meaning the number of VaR violations matched the expected probability.\n- Although Christoffersen rejected all models (indicating clustered violations), VaRPot still outperformed others.\n- VaRPot was more stable than VaREWMA, which dropped from Green in 2007 to Yellow in 2008.\n- Other models (VaRBHS, VaRn, VaRt) performed poorly:\n  - Basel Red Zones in both years, meaning excessive violations.\n  - Kupiec Test Rejected, suggesting they either underestimated or overestimated risk.\n  - Christoffersen Test Rejected, confirming violation clustering.\n\n**Final Choice for VaR: VaRPot** (Reliable across years, Basel Green, Kupiec Accepted).\n\n**Best ES Model: ESEWMA**\n- ESEWMA remai